In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm


In [3]:
print("started loading complaintsinitial.csv at",datetime.now())
df_initial = pd.read_csv('complaintsinitial.csv') #1.27 GB 
print("finished loading complaintsinitial.csv at",datetime.now())  #15 seconds to load

print(df_initial.shape) # 1741869 rows, 18 columns

#get number of unique values per column
unique_values_per_column = df_initial.nunique();

print("Number of unique values per column:-\n" , unique_values_per_column);
print("\n\n")

#get number of missing values per column
nan_counts_per_column = df_initial.isna().sum()
print("Number of missing values per column:-\n" , nan_counts_per_column );

#The following rows are missing 0 values (all categorical variables) - Product (9 unique), Issue(81 unique), Company(4735 unique), Submitted via(7 unique), Timely response(2 unique)

#sub-product missing 93 values out of 1741869 so will remove these rows as we might want to include it as a predictor variable since it only has 48 unique values 
df_initial = df_initial[~df_initial['Sub-product'].isna()]

#drop the following columns
#Consumer complaint narrative - free text
#Tags - 1583239 missing out of 1741869 total
#Date sent to company. Over 92% are the same day as date received and of the remaining 8% almost all are within 1 week so high collinearity
# Zip codes as these have high cardinality, which could lead to overfitting. We have enough other variables to ensure we don't underfit. Also a number missing or have XX
# Complaint ID is unique per complaint so also has high cardinality
#Customer disputed? has no data

df_Group6 = df_initial.drop(['Consumer complaint narrative','Tags','Date sent to company','ZIP code','Complaint ID','Consumer disputed?'],axis=1)

del df_initial # remove from memory

started loading complaintsinitial.csv at 2024-04-23 11:07:48.884041
finished loading complaintsinitial.csv at 2024-04-23 11:08:05.321565
(1741869, 18)
Number of unique values per column:-
 Date received                      1096
Product                               9
Sub-product                          48
Issue                                81
Sub-issue                           167
Consumer complaint narrative     583626
Company public response              10
Company                            4735
State                                61
ZIP code                          23645
Tags                                  3
Consumer consent provided?            4
Submitted via                         7
Date sent to company               1171
Company response to consumer          5
Timely response?                      2
Consumer disputed?                    0
Complaint ID                    1741869
dtype: int64



Number of missing values per column:-
 Date received                       

In [4]:
def CompanyResponse(df):
    # Create a new column 'Closed with monetary relief' and set it to 1 where 'Company response to consumer' equals 'Closed with monetary relief', 
    # else set it to 0
    df['Closed with monetary relief'] = (df['Company response to consumer'] == 'Closed with monetary relief').astype(int)
    return df


In [5]:
def GetValueCounts(df, predictor_var):
    print("\n")
    print(f"Count of variables for predictor variable '{predictor_var}':")
    print(df[predictor_var].value_counts().head(10))

In [6]:
#set 'Closed with monetary relief' variable to 1 where 'Company response to consumer' equals 'Closed with monetary relief' else 0. This is the predictor variable for our models
CompanyResponse(df_Group6)  

# The following code shows the distribution of classes in our predictors and response variable. Specifically we see that only 43966 (just over 2.5%) are marked with 'Closed with monetary relief while 1697810 out of 1741776 (almost 97.5%) are marked as other

for predictor_var in ('Closed with monetary relief','Product', 'Issue'):
    GetValueCounts(df_Group6,predictor_var)




Count of variables for predictor variable 'Closed with monetary relief':
Closed with monetary relief
0    1697810
1      43966
Name: count, dtype: int64


Count of variables for predictor variable 'Product':
Product
Credit reporting, credit repair services, or other personal consumer reports    1196468
Debt collection                                                                  183830
Credit card or prepaid card                                                      105629
Checking or savings account                                                       91356
Mortgage                                                                          74489
Money transfer, virtual currency, or money service                                35757
Vehicle loan or lease                                                             23463
Student loan                                                                      16477
Payday loan, title loan, or personal loan                                     

In [15]:
Y=df_Group6['Closed with monetary relief']  #response variable

# Need numeric values for logistic regression. Do in this order to avoid warnings
X = df_Group6[['Product', 'Issue']].copy()
X['ProductCodes'] = X['Product'].astype('category').cat.codes
X['IssueCodes'] = X['Issue'].astype('category').cat.codes

# Split data into training and testing sets (splitting 80:20 train:test) 

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=21)

# First we fit a logistic regression model with product codes & response variable 'Closed with monetary relief' (1 or 0)

model1 = sm.Logit(y_train, x_train['ProductCodes']).fit()
print("The confusion matrix (training set) for this model is\n", confusion_matrix(y_train, model1.predict(x_train['ProductCodes'])>0.5))
print("The confusion matrix (testing set) for this model is\n", confusion_matrix(y_test, model1.predict(x_test['ProductCodes'])>0.5))

print("\nModel 1 summary statistics:\n")
print(model1.summary2())



Optimization terminated successfully.
         Current function value: 0.131544
         Iterations 8
The confusion matrix (training set) for this model is
 [[1358281       0]
 [  35139       0]]
The confusion matrix (testing set) for this model is
 [[339529      0]
 [  8827      0]]

Model 1 summary statistics:

                               Results: Logit
Model:              Logit                       Method:           MLE        
Dependent Variable: Closed with monetary relief Pseudo R-squared: -0.118     
Date:               2024-04-23 11:38            AIC:              366594.8939
No. Observations:   1393420                     BIC:              366607.0411
Df Model:           0                           Log-Likelihood:   -1.8330e+05
Df Residuals:       1393419                     LL-Null:          -1.6401e+05
Converged:          1.0000                      LLR p-value:      nan        
No. Iterations:     8.0000                      Scale:            1.0000     
---------------

In [16]:
# Now we fit a logistic regression model with product codes and issue codes as predictor variables

model2 = sm.Logit(y_train, x_train[['ProductCodes','IssueCodes']]).fit()
print("The confusion matrix (training set) for this model is\n", confusion_matrix(y_train, model2.predict(x_train[['ProductCodes','IssueCodes']])>0.5))
print("The confusion matrix (testing set) for this model is\n", confusion_matrix(y_test, model2.predict(x_test[['ProductCodes','IssueCodes']])>0.5))


print("\nModel 2 summary statistics:\n")
print(model2.summary2())

Optimization terminated successfully.
         Current function value: 0.119301
         Iterations 8
The confusion matrix (training set) for this model is
 [[1358281       0]
 [  35139       0]]
The confusion matrix (testing set) for this model is
 [[339529      0]
 [  8827      0]]

Model 2 summary statistics:

                               Results: Logit
Model:              Logit                       Method:           MLE        
Dependent Variable: Closed with monetary relief Pseudo R-squared: -0.014     
Date:               2024-04-23 11:39            AIC:              332477.5024
No. Observations:   1393420                     BIC:              332501.7969
Df Model:           1                           Log-Likelihood:   -1.6624e+05
Df Residuals:       1393418                     LL-Null:          -1.6401e+05
Converged:          1.0000                      LLR p-value:      1.0000     
No. Iterations:     8.0000                      Scale:            1.0000     
---------------

In [14]:
# We choose the second model for its lower AIC. However both models cannot predict any positive cases ('Closed with monetary relief') because there are just over 2.5% of these in the dataset. 
#We will test different thresholds to see which has the best balance between True Positive Rate and True Negative Rate. 

ROC_df = pd.DataFrame({"threshold": [], 
                       "Training TPR": [], 
                       "Training TNR": [], 
                       "Test TPR": [], 
                       "Test TNR": []})

for threshold in np.arange(0, 0.5, 0.01): 
    predictions_train = (model2.predict(x_train[['ProductCodes','IssueCodes']].astype(float)) > threshold).astype(int)
    predictions_test = (model2.predict(x_test[['ProductCodes','IssueCodes']].astype(float)) > threshold).astype(int)
    tn1, fp1, fn1, tp1 = confusion_matrix(y_train, predictions_train).ravel()
    tn2, fp2, fn2, tp2 = confusion_matrix(y_test, predictions_test).ravel()
    ROC_df.loc[-1] = [threshold, tp1 / (tp1 + fn1), tn1 / (tn1 + fp1), tp2 / (tp2 + fn2), tn2 / (tn2 + fp2)]
    ROC_df.index = ROC_df.index + 1
    ROC_df = ROC_df.sort_index()

print(ROC_df.head(50))
print(ROC_df.tail(12))

    threshold  Training TPR  Training TNR  Test TPR  Test TNR
0        0.49      0.000000      1.000000  0.000000  1.000000
1        0.48      0.000000      1.000000  0.000000  1.000000
2        0.47      0.000000      1.000000  0.000000  1.000000
3        0.46      0.000000      1.000000  0.000000  1.000000
4        0.45      0.000000      1.000000  0.000000  1.000000
5        0.44      0.000000      1.000000  0.000000  1.000000
6        0.43      0.037366      0.993700  0.032174  0.993512
7        0.42      0.037366      0.993700  0.032174  0.993512
8        0.41      0.037366      0.993700  0.032174  0.993512
9        0.40      0.037366      0.993700  0.032174  0.993512
10       0.39      0.037366      0.993700  0.032174  0.993512
11       0.38      0.037366      0.993700  0.032174  0.993512
12       0.37      0.037366      0.993700  0.032174  0.993512
13       0.36      0.037480      0.993684  0.032174  0.993497
14       0.35      0.037480      0.993684  0.032174  0.993497
15      

We chose 0.10 as our threshold, because it has the best balance between True Positive Rate and True Negative Rate.

In [21]:
# Using the test dataset predict the product using model 2 with a 0.1 threshold.

threshold = 0.10
predictions = (model2.predict(x_test[['ProductCodes','IssueCodes']].astype(float)) > threshold).astype(int)

results_all_df = pd.DataFrame({'Actual': y_test, 'Predicted_Probability': predictions, 'Product': x_test['ProductCodes'], 'Issue': x_test['IssueCodes']})

# Define a dictionary mapping product codes to product names
product_code_to_name = {code: name for code, name in zip(x_test['ProductCodes'], x_test['Product'])}

# Define a dictionary mapping product codes to product names
state_code_to_name = {code: name for code, name in zip(x_test['IssueCodes'], x_test['Issue'])}


# Map product codes to product names in the DataFrame
results_all_df['Product'] = results_all_df['Product'].map(product_code_to_name)

# Map product codes to product names in the DataFrame
results_all_df['Issue'] = results_all_df['Issue'].map(state_code_to_name)

results_all_df.head()


results_all_df['Product_Issue'] = results_all_df['Product'] + '_' + results_all_df['Issue']

#Count the occurrences of each value in the 'Product' table
product_issue_counts = results_all_df['Product_Issue'].value_counts()

positive_outcomes_df = results_all_df[results_all_df['Actual'] == 1]

# Count the occurrences of a positive outcome ('Closed with monetary relief' = 1) for each value in the 'Product' table
positive_outcomes_counts = positive_outcomes_df['Product_Issue'].value_counts()
positive_outcomes_counts = positive_outcomes_counts.sort_values(ascending=False)

#print(positive_outcomes_counts.head(10))
#print("\nThe Product & Issue combination which has the highest predicted number of 'Closed with monetary relief' is: '" + positive_outcomes_counts.index[0] + "' at " + str(positive_outcomes_counts.iloc[0]))

# Divide positive outcomes counts by product counts to get probability of each product value
output_per_value = positive_outcomes_counts / product_issue_counts

# Display the output
#print("\nWith a threshold of " + str(threshold) + " the probability each Product category is 'Closed with monetary relief' in descending order:\n")
output_per_value = output_per_value.sort_values(ascending=False)
#print(output_per_value)
print("\nWith a threshold of " + str(threshold) + " the Product & Issue with the highest predicted percentage 'Closed with monetary relief' is Product_Issue:'" + output_per_value.index[0] + "' which has a probability of: " + str(round(output_per_value.iloc[0],3)))


#Getting counts of individual Product and Issue from the predicted dataset
results_all_df['Product'] = results_all_df['Product'] 
results_all_df['Issue'] = results_all_df['Issue']
positive_outcomes_product_counts = positive_outcomes_df['Product'].value_counts().fillna(0)
positive_outcomes_product_counts =positive_outcomes_product_counts.sort_values(ascending=False)
product_counts = results_all_df['Product'].value_counts()
product_test_probability = positive_outcomes_product_counts/product_counts
product_test_probability = product_test_probability.sort_values(ascending=False).fillna(0)

issue_counts = results_all_df['Issue'].value_counts()
positive_outcomes_issue_counts = positive_outcomes_df['Issue'].value_counts().fillna(0)
positive_outcomes_issue_counts = positive_outcomes_issue_counts.sort_values(ascending=False)
issue_test_probability = positive_outcomes_issue_counts/issue_counts
issue_test_probability = issue_test_probability.sort_values(ascending=False).fillna(0)

#print("\nCount of Product in test dataset:")
#print(results_all_df['Product'].head())
#print(results_all_df['Product'].value_counts())

print("\nWith a threshold of " + str(threshold) + " count of Product 'Closed with monetary relief' in test dataset:")
print(positive_outcomes_product_counts)
print("\nWith a threshold of " + str(threshold) + " probability of Product 'Closed with monetary relief' in test dataset:")
print(product_test_probability)

product_test_probability.to_csv('Product_probability.csv')

#print("\nCount of Issue in test dataset:")
#print(results_all_df['Product'].head())
#print(results_all_df['Issue'].value_counts())

#print("\nCount of Issue 'Closed with monetary relief' in test dataset:")
#print(positive_outcomes_issue_counts)
#print("\nProbability of Issue 'Closed with monetary relief' in test dataset:")
#print(issue_test_probability)

#print("\n Top 2 Results of Product:" + results_all_df['Product'].index[0])  #.head(2))


#print(positive_outcomes_product_counts)
#print("The total predicted Product number in the test set 'Closed with monetary relief' is: " + positive_outcomes_product_counts) #+ " which has an overall probability of "  + (positive_outcomes_product_counts/product_counts))





With a threshold of 0.1 the Product & Issue with the highest predicted percentage 'Closed with monetary relief' is Product_Issue:'Credit card or prepaid card_Fees or interest' which has a probability of: 0.364

With a threshold of 0.1 count of Product 'Closed with monetary relief' in test dataset:
Product
Credit card or prepaid card                                                     3752
Checking or savings account                                                     2930
Money transfer, virtual currency, or money service                               695
Mortgage                                                                         574
Credit reporting, credit repair services, or other personal consumer reports     332
Debt collection                                                                  215
Payday loan, title loan, or personal loan                                        139
Vehicle loan or lease                                                            137
Student loan

In [21]:
# Unique values of States Column

states_list = df_Group6['State']
states_list.unique()

array(['OH', 'TN', 'FL', 'PA', 'CA', 'TX', 'VA', 'MO', 'GA', 'NY', 'CO',
       'WA', 'MN', 'AL', 'IL', 'NC', 'OK', 'MD', 'MI', 'LA', 'SC', nan,
       'OR', 'NH', 'IN', 'NJ', 'DC', 'UT', 'KY', 'NV', 'MA', 'AA', 'RI',
       'KS', 'DE', 'WI', 'MS', 'NM', 'PR', 'NE', 'CT', 'AR', 'AZ', 'IA',
       'ME', 'HI', 'WV', 'ID', 'ND', 'MT',
       'UNITED STATES MINOR OUTLYING ISLANDS', 'AE', 'WY', 'GU', 'SD',
       'AK', 'AP', 'VI', 'VT', 'MP', 'AS', 'MH'], dtype=object)

In [22]:
# Dictionary of State initials and Full Name

state_names = {
    'OH': 'Ohio',
    'TN': 'Tennessee',
    'FL': 'Florida',
    'PA': 'Pennsylvania',
    'CA': 'California',
    'TX': 'Texas',
    'VA': 'Virginia',
    'MO': 'Missouri',
    'GA': 'Georgia',
    'NY': 'New York',
    'CO': 'Colorado',
    'WA': 'Washington',
    'MN': 'Minnesota',
    'AL': 'Alabama',
    'IL': 'Illinois',
    'NC': 'North Carolina',
    'OK': 'Oklahoma',
    'MD': 'Maryland',
    'MI': 'Michigan',
    'LA': 'Louisiana',
    'SC': 'South Carolina',
    'nan': 'Unknown',
    'OR': 'Oregon',
    'NH': 'New Hampshire',
    'IN': 'Indiana',
    'NJ': 'New Jersey',
    'DC': 'Washington DC',
    'UT': 'Utah',
    'KY': 'Kentucky',
    'NV': 'Nevada',
    'MA': 'Massachusetts',
    'AA': 'Armed Forces America',
    'RI': 'Rhode Island',
    'KS': 'Kansas',
    'DE': 'Delaware',
    'WI': 'Wisconsin',
    'MS': 'Mississippi',
    'NM': 'New Mexico',
    'PR': 'Puerto Rico',
    'NE': 'Nebraska',
    'CT': 'Connecticut',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'IA': 'Iowa',
    'ME': 'Maine',
    'HI': 'Hawaii',
    'WV': 'West Virginia',
    'ID': 'Idaho',
    'ND': 'North Dakota',
    'MT': 'Montana',
    'UNITED STATES MINOR OUTLYING ISLANDS':'UNITED STATES MINOR OUTLYING ISLANDS',
    'AE': 'Armed Forces',
    'WY': 'Wyoming',
    'GU': 'Guam',
    'SD': 'South Dakota',
    'AK': 'Alaska',
    'AP': 'Armed Forces Pacific',
    'VI': 'Virgin Islands',
    'VT': 'Vermont',
    'MP': 'Northern Mariana Islands',
    'AS': 'American Samoa',
    'MH': 'Marshall Islands'
}

In [23]:
# Replace state initials with full names
df_Group6['State']=states_list.map(state_names)

In [24]:
# Exporting updated dataset to use in Tableau

df_Group6.to_csv('Group6-TableauExport.csv')